In [1]:
# Célula 1: Configuração e Importações

# Importação de bibliotecas essenciais
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

# Importações para TensorFlow/Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
# from tensorflow.keras.metrics import Precision, Recall, F1Score # Para métricas personalizadas se necessário

# Importações para Machine Learning (avaliação)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler # Pode ser útil se as features não foram padronizadas antes
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, auc
from joblib import dump, load # Para salvar e carregar dados e modelos

# Para a camada TCN, usaremos uma implementação comum.
# Se você não tiver a biblioteca 'keras-tcn' instalada, ela precisará ser instalada:
# !pip install keras-tcn 
# (Nota: A TCN será implementada manualmente abaixo, então a biblioteca keras-tcn não é estritamente necessária
# para esta versão do código, mas a importação está mantida caso você queira alternar implementações)
try:
    from tcn import TCN #
    from tensorflow.keras.utils import get_custom_objects #
    # get_custom_objects().update({'TCN': TCN}) # Versões antigas
except ImportError:
    print("Biblioteca 'keras-tcn' não encontrada. A implementação manual da TCN será usada.") #

# Configurações de exibição do Pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Desativar avisos
import warnings
warnings.filterwarnings('ignore')

print("Bibliotecas importadas com sucesso.")

# Definir caminho para os dados pré-processados
DATA_DIR = 'processed_data' 

# Nome do arquivo onde as sequências e rótulos foram salvos
SEQUENCES_FILE = 'model2_sequences_labels.pkl' #

2025-05-22 23:28:02.381484: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-22 23:28:02.389975: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747967282.399731  474636 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747967282.402532  474636 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747967282.410386  474636 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Bibliotecas importadas com sucesso.


In [2]:
# Célula 2: Carregamento, Pré-processamento e Divisão dos Dados

print("Carregando dados de sequências pré-processados...") #

try:
    # Carregar o dicionário contendo 'sequences' e 'labels'
    loaded_data = load(os.path.join(DATA_DIR, SEQUENCES_FILE))
    
    X = loaded_data['sequences'] # As sequências de características (num_samples, timesteps, num_features)
    y = loaded_data['labels']   # Os rótulos de fadiga (0 ou 1) para cada sequência

    print(f"Dados carregados.")
    print(f"Formato das sequências (X): {X.shape}") #
    print(f"Formato dos rótulos (y): {y.shape}") #
    
    unique_labels, counts = np.unique(y, return_counts=True)
    label_distribution = dict(zip(unique_labels, counts))
    print(f"Distribuição dos rótulos de fadiga: {label_distribution}") #
    if 1 in unique_labels:
        print(f"Porcentagem da classe 'Fadigado' (1): {counts[counts.tolist().index(1)] / np.sum(counts):.2%}") #
    else:
        print("Classe 'Fadigado' (1) não encontrada.") #


except FileNotFoundError:
    print(f"Erro: Arquivo '{SEQUENCES_FILE}' não encontrado em '{DATA_DIR}'.") #
    print("Por favor, certifique-se de que o notebook '00_Data_Preparation_and_Exploration.ipynb' foi executado corretamente.") #
    # exit()
except Exception as e:
    print(f"Ocorreu um erro ao carregar ou processar os dados: {e}")
    # exit()

print("\nDividindo o dataset em conjuntos de treino, validação e teste...")

# Divisão estratificada para manter a proporção das classes
# 1. Divisão inicial: 80% treino + validação, 20% teste
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 2. Divisão do conjunto de treino+validação: 80% treino, 20% validação
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.2, random_state=42, stratify=y_train_val
)

print(f"Tamanho do conjunto de Treino: X={X_train.shape}, y={y_train.shape}") #
print(f"Tamanho do conjunto de Validação: X={X_val.shape}, y={y_val.shape}") #
print(f"Tamanho do conjunto de Teste: X={X_test.shape}, y={y_test.shape}") #

# Padronização das características
# X tem shape (num_samples, timesteps, num_features)
n_samples, n_timesteps, n_features = X_train.shape #

# Achatar para (num_samples * timesteps, num_features)
X_train_reshaped = X_train.reshape(-1, n_features)
X_val_reshaped = X_val.reshape(-1, n_features)
X_test_reshaped = X_test.reshape(-1, n_features)

print("\nPadronizando as características (StandardScaler)...")
scaler = StandardScaler()

X_train_scaled_reshaped = scaler.fit_transform(X_train_reshaped)
X_val_scaled_reshaped = scaler.transform(X_val_reshaped) #
X_test_scaled_reshaped = scaler.transform(X_test_reshaped)

# Reformatar de volta para (num_samples, timesteps, num_features)
X_train_scaled = X_train_scaled_reshaped.reshape(n_samples, n_timesteps, n_features)
X_val_scaled = X_val_scaled_reshaped.reshape(X_val.shape[0], n_timesteps, n_features)
X_test_scaled = X_test_scaled_reshaped.reshape(X_test.shape[0], n_timesteps, n_features)

print("Padronização concluída.")
print("Dados prontos para o treinamento do Modelo (CNN+TCN).")

Carregando dados de sequências pré-processados...
Dados carregados.
Formato das sequências (X): (11006, 90, 2)
Formato dos rótulos (y): (11006,)
Distribuição dos rótulos de fadiga: {0: 6889, 1: 4117}
Ocorreu um erro ao carregar ou processar os dados: 1 is not in list

Dividindo o dataset em conjuntos de treino, validação e teste...
Tamanho do conjunto de Treino: X=(7043, 90, 2), y=(7043,)
Tamanho do conjunto de Validação: X=(1761, 90, 2), y=(1761,)
Tamanho do conjunto de Teste: X=(2202, 90, 2), y=(2202,)

Padronizando as características (StandardScaler)...
Padronização concluída.
Dados prontos para o treinamento do Modelo (CNN+TCN).


In [2]:
# Célula 3: Implementação e Treinamento da CNN+TCN

print("--- Implementando e Treinando o Modelo (CNN + TCN Manual Robusta) ---")

# --- Parâmetros da CNN ---
NUM_CNN_FILTERS = 64  # Número de filtros para as camadas CNN
CNN_KERNEL_SIZE = 3   # Tamanho do kernel para as camadas CNN
CNN_DROPOUT_RATE = 0.25 # Taxa de Dropout para camadas CNN (geralmente menor que TCN)

# --- Parâmetros da TCN ---
NUM_TCN_BLOCKS = 3     # Número de blocos residuais TCN (ajustado, pode ser 2-4)
NUM_FILTERS_TCN = 64   # Número de filtros convolucionais em cada camada TCN
KERNEL_SIZE_TCN = 2    # Tamanho do kernel para as convoluções TCN
TCN_DROPOUT_RATE = 0.5 # Taxa de Dropout para TCN
LEARNING_RATE = 0.001  # Taxa de aprendizado do otimizador

from tensorflow.keras import regularizers

# --- Definição do Bloco Residual TCN (Manual Robusto) ---
def residual_block_manual(x, nb_filters, kernel_size, dilation_rate, dropout_rate, name='tcn_block'):
    """
    Constrói um bloco residual da TCN utilizando apenas camadas Keras nativas.
    Incorpora duas convoluções dilatadas, Batch Normalization, ativação ReLU, Dropout
    e uma conexão residual (skip connection), com regularização L2.
    """
    input_to_block = x

    # Bloco convolucional 1 com regularização L2
    conv1 = layers.Conv1D(
        filters=nb_filters,
        kernel_size=kernel_size,
        dilation_rate=dilation_rate,
        padding='causal',
        kernel_regularizer=regularizers.l2(0.0005), #
        name=f'{name}_conv1_d{dilation_rate}'
    )(x)
    conv1 = layers.BatchNormalization(name=f'{name}_bn1_d{dilation_rate}')(conv1)
    conv1 = layers.Activation('relu', name=f'{name}_relu1_d{dilation_rate}')(conv1)
    conv1 = layers.Dropout(dropout_rate, name=f'{name}_dropout1_d{dilation_rate}')(conv1)

    # Bloco convolucional 2 com regularização L2
    conv2 = layers.Conv1D(
        filters=nb_filters,
        kernel_size=kernel_size,
        dilation_rate=dilation_rate,
        padding='causal',
        kernel_regularizer=regularizers.l2(0.0005), #
        name=f'{name}_conv2_d{dilation_rate}'
    )(conv1)
    conv2 = layers.BatchNormalization(name=f'{name}_bn2_d{dilation_rate}')(conv2)
    conv2 = layers.Activation('relu', name=f'{name}_relu2_d{dilation_rate}')(conv2)
    conv2 = layers.Dropout(dropout_rate, name=f'{name}_dropout2_d{dilation_rate}')(conv2)

    # Conexão residual
    if input_to_block.shape[-1] != nb_filters:
        input_to_block = layers.Conv1D(
            filters=nb_filters,
            kernel_size=1,
            padding='same',
            kernel_regularizer=regularizers.l2(0.0005), #
            name=f'{name}_skip_conv_d{dilation_rate}'
        )(input_to_block)
    
    output = layers.add([input_to_block, conv2], name=f'{name}_add_d{dilation_rate}') #
    output = layers.Activation('relu', name=f'{name}_final_relu_d{dilation_rate}')(output) #
    return output


# --- Definindo a arquitetura completa da CNN+TCN ---
input_shape_dims = (X_train_scaled.shape[1], X_train_scaled.shape[2])
inputs = keras.Input(shape=input_shape_dims)
x = inputs

# --- Camadas CNN ---
# Bloco CNN 1
x = layers.Conv1D(filters=NUM_CNN_FILTERS, kernel_size=CNN_KERNEL_SIZE, padding='same', 
                  kernel_regularizer=regularizers.l2(0.0005), name='cnn_conv1')(x)
x = layers.BatchNormalization(name='cnn_bn1')(x)
x = layers.Activation('relu', name='cnn_relu1')(x)
x = layers.Dropout(CNN_DROPOUT_RATE, name='cnn_dropout1')(x)
# Opcional: MaxPooling1D. Se usar, cuidado com o encurtamento da sequência.
# x = layers.MaxPooling1D(pool_size=2, name='cnn_pool1')(x) 

# (Opcional) Bloco CNN 2
# x = layers.Conv1D(filters=NUM_CNN_FILTERS, kernel_size=CNN_KERNEL_SIZE, padding='same', 
#                   kernel_regularizer=regularizers.l2(0.0005), name='cnn_conv2')(x)
# x = layers.BatchNormalization(name='cnn_bn2')(x)
# x = layers.Activation('relu', name='cnn_relu2')(x)
# x = layers.Dropout(CNN_DROPOUT_RATE, name='cnn_dropout2')(x)

# --- Camadas TCN ---
# A saída 'x' das camadas CNN agora alimenta as camadas TCN
for i in range(NUM_TCN_BLOCKS):
    dilation_rate = 2 ** i
    x = residual_block_manual(x, NUM_FILTERS_TCN, KERNEL_SIZE_TCN, dilation_rate, TCN_DROPOUT_RATE, name=f'tcn_block_{i}')

# --- Camada de Saída ---
x = layers.GlobalAveragePooling1D(name='global_avg_pooling')(x)
outputs = layers.Dense(1, activation='sigmoid', name='output_layer')(x)

model_cnn_tcn = keras.Model(inputs=inputs, outputs=outputs, name='CNN_TCN_Manual_Model')

# --- Compilação do Modelo ---
model_cnn_tcn.compile(optimizer=optimizers.Adam(learning_rate=LEARNING_RATE),
                      loss='binary_crossentropy',
                      metrics=['accuracy'])

model_cnn_tcn.summary()

# --- Callbacks para Treinamento ---
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True) #

MODEL_SAVE_PATH_CNN_TCN = os.path.join(DATA_DIR, 'best_model_cnn_tcn.keras') # Nome do arquivo atualizado

model_checkpoint = ModelCheckpoint(
    MODEL_SAVE_PATH_CNN_TCN,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

# Cálculo de pesos para lidar com desbalanceamento de classes
class_weights = {}
unique_labels_train, counts_train = np.unique(y_train, return_counts=True)
total_samples_train = len(y_train)

if len(counts_train) == 2: # Garante que ambas as classes estão presentes
    class_weights[0] = total_samples_train / (2.0 * counts_train[0])
    class_weights[1] = total_samples_train / (2.0 * counts_train[1])
    print(f"\nPesos de Classe Calculados: {class_weights}")
else:
    print("\nNão foi possível calcular os pesos de classe (uma classe pode estar faltando no y_train). Usando pesos None.")
    class_weights = None


print("\nIniciando treinamento do Modelo (CNN+TCN Manual Robusta)...")

# --- Treinamento do Modelo ---
history = model_cnn_tcn.fit(
    X_train_scaled, y_train,
    epochs=200, # Pode precisar ajustar
    batch_size=32, #
    validation_data=(X_val_scaled, y_val),
    callbacks=[early_stopping, model_checkpoint],
    class_weight=class_weights,
    verbose=1
)

print("\nTreinamento da CNN+TCN Manual Robusta concluído.")

# --- Plotar histórico de treinamento ---
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Acurácia de Treino')
plt.plot(history.history['val_accuracy'], label='Acurácia de Validação')
plt.title('Acurácia do Modelo CNN+TCN ao longo das Épocas')
plt.xlabel('Época')
plt.ylabel('Acurácia')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Perda de Treino')
plt.plot(history.history['val_loss'], label='Perda de Validação')
plt.title('Perda do Modelo CNN+TCN ao longo das Épocas')
plt.xlabel('Época')
plt.ylabel('Perda')
plt.legend()
plt.tight_layout()
plt.show()

# Carregar os melhores pesos salvos pelo ModelCheckpoint
try:
    # Nota: Keras salva o modelo inteiro (arquitetura + pesos) no formato .keras
    # Não é necessário registrar custom objects para a implementação manual.
    model_cnn_tcn = keras.models.load_model(MODEL_SAVE_PATH_CNN_TCN)
    print(f"Melhor modelo CNN+TCN carregado de: {MODEL_SAVE_PATH_CNN_TCN}")
except Exception as e:
    print(f"Não foi possível carregar o melhor modelo CNN+TCN: {e}. Usando o modelo final do treinamento.") #

print("\nTreinamento do Modelo CNN+TCN finalizado. Próximo passo: Avaliação do Modelo.")

--- Implementando e Treinando o Modelo (CNN + TCN Manual Robusta) ---


NameError: name 'X_train_scaled' is not defined

In [1]:
# Célula 4: Avaliação do Modelo no Conjunto de Teste

print("--- Avaliando o Modelo (CNN+TCN) no Conjunto de Teste ---")

# model_cnn_tcn deve conter o modelo com os melhores pesos de validação, carregado no final da Célula 3.

# 1. Avaliar o modelo no conjunto de teste [cite: 43]
loss_test, accuracy_test = model_cnn_tcn.evaluate(X_test_scaled, y_test, verbose=0)
print(f"\nResultados no Conjunto de Teste:")
print(f"  Perda (Loss): {loss_test:.4f}")
print(f"  Acurácia (Accuracy): {accuracy_test:.4f}")

# 2. Fazer previsões no conjunto de teste
y_pred_proba = model_cnn_tcn.predict(X_test_scaled).ravel() # Probabilidades
y_pred = (y_pred_proba > 0.5).astype(int)             # Classes (0 ou 1)

# 3. Calcular Métricas de Classificação Detalhadas
print("\nMétricas de Classificação Detalhadas:")
print(f"  Acurácia Geral: {accuracy_score(y_test, y_pred):.4f}")
print(f"  Precisão (Precision): {precision_score(y_test, y_pred, zero_division=0):.4f}")
print(f"  Recall (Sensibilidade): {recall_score(y_test, y_pred, zero_division=0):.4f}")
print(f"  F1-Score: {f1_score(y_test, y_pred, zero_division=0):.4f}")

# 4. Matriz de Confusão
cm = confusion_matrix(y_test, y_pred)
print("\nMatriz de Confusão:")
print(cm)

# Visualizar a Matriz de Confusão
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Não Fadigado (0)', 'Fadigado (1)'], # [cite: 44]
            yticklabels=['Não Fadigado (0)', 'Fadigado (1)']) # [cite: 44]
plt.xlabel('Previsão')
plt.ylabel('Real')
plt.title('Matriz de Confusão do Modelo TCN')
plt.show()

# 5. Curva ROC e AUC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(7, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'Curva ROC (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Classificador Aleatório')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taxa de Falsos Positivos (FPR)')
plt.ylabel('Taxa de Verdadeiros Positivos (TPR)')
plt.title('Curva ROC do Modelo CNN+TCN')
plt.legend(loc="lower right")
plt.show()

print("\nAvaliação do Modelo TCN concluída.") # [cite: 45]

--- Avaliando o Modelo (CNN+TCN) no Conjunto de Teste ---


NameError: name 'model_cnn_tcn' is not defined

In [5]:
# Célula 5: Salvar o Modelo Final e os Pré-processadores

print("--- Salvando o Modelo CNN+TCN e os Pré-processadores ---")

# 1. Salvar o Modelo CNN+TCN
# MODEL_SAVE_PATH_CNN_TCN já está definido na Célula 3 e o melhor modelo já foi salvo.
# A linha no final da Célula 3 já deveria ter carregado essa melhor versão em 'model_cnn_tcn'.
print(f"O melhor modelo CNN+TCN já foi salvo em: {MODEL_SAVE_PATH_CNN_TCN}") # [cite: 46]

# Se você quiser salvar o modelo 'model_cnn_tcn' novamente (o estado atual após o treinamento
# e potencial restauração pelo EarlyStopping), pode fazer assim:
# model_cnn_tcn.save(MODEL_SAVE_PATH_CNN_TCN)
# print(f"Modelo CNN+TCN salvo novamente em: {MODEL_SAVE_PATH_CNN_TCN}")


# 2. Salvar o StandardScaler usado para as features
SCALER_SAVE_PATH = os.path.join(DATA_DIR, 'scaler_model_cnn_tcn.joblib') # Nome do arquivo atualizado

try:
    dump(scaler, SCALER_SAVE_PATH)
    print(f"StandardScaler salvo em: {SCALER_SAVE_PATH}")
except NameError:
    print("Erro: 'scaler' não está definido. Certifique-se de que a Célula 2 foi executada corretamente.") # [cite: 47]
except Exception as e:
    print(f"Erro ao salvar o StandardScaler: {e}") # [cite: 47]


# 3. Salvar o LabelEncoder (se usado)
# Neste caso, os rótulos são 0 e 1, então não é estritamente necessário. [cite: 48]
print("Seus rótulos (y) já eram numéricos (0/1). Não é necessário salvar um LabelEncoder.") # [cite: 49]

print("\nSalvamento do modelo CNN+TCN e pré-processadores concluído. O modelo está pronto para inferência.")

--- Salvando o Modelo CNN+TCN e os Pré-processadores ---
O melhor modelo CNN+TCN já foi salvo em: processed_data/best_model_cnn_tcn.keras
StandardScaler salvo em: processed_data/scaler_model_cnn_tcn.joblib
Seus rótulos (y) já eram numéricos (0/1). Não é necessário salvar um LabelEncoder.

Salvamento do modelo CNN+TCN e pré-processadores concluído. O modelo está pronto para inferência.


In [1]:
# Célula de Teste do Modelo CNN-TCN com Câmera em Tempo Real (usando EAR e MAR)

import cv2
import numpy as np
import mediapipe as mp
from tensorflow import keras
from joblib import load
import os
from collections import deque
import math # Para a função de distância

print("--- Testando o Modelo CNN-TCN com Câmera ---")

# --- 0. Constantes e Caminhos (ajuste conforme necessário) ---
DATA_DIR = 'processed_data' # Deve ser o mesmo diretório onde você salvou o modelo e o scaler
MODEL_FILE = 'best_model_cnn_tcn.keras' # Nome do arquivo do seu modelo TCN treinado
SCALER_FILE = 'scaler_model_cnn_tcn.joblib' # Nome do arquivo do seu scaler para o modelo TCN

MODEL_PATH = os.path.join(DATA_DIR, MODEL_FILE)
SCALER_PATH = os.path.join(DATA_DIR, SCALER_FILE)

# Tentar carregar o modelo e o scaler
try:
    # A variável agora se chama 'model_cnn_tcn' para clareza
    model_cnn_tcn = keras.models.load_model(MODEL_PATH)
    print(f"Modelo CNN-TCN carregado de: {MODEL_PATH}")
except Exception as e:
    print(f"Erro ao carregar o modelo: {e}")
    print("Certifique-se de que o caminho e o nome do arquivo do modelo TCN estão corretos.")
    exit()

try:
    scaler = load(SCALER_PATH)
    print(f"StandardScaler carregado de: {SCALER_PATH}")
except Exception as e:
    print(f"Erro ao carregar o scaler: {e}")
    print("Certifique-se de que o caminho e o nome do arquivo do scaler TCN estão corretos.")
    exit()

# --- 1. Obter Parâmetros do Modelo Carregado ---
try:
    # A forma da entrada do modelo deve ser (None, TIMESTEPS, NUM_FEATURES_MODEL)
    _, TIMESTEPS, NUM_FEATURES_MODEL = model_cnn_tcn.input_shape
    print(f"Configuração do modelo: TIMESTEPS={TIMESTEPS}, NUM_FEATURES_MODEL={NUM_FEATURES_MODEL}")
    if NUM_FEATURES_MODEL != 2:
        print(f"AVISO CRÍTICO: O modelo TCN espera {NUM_FEATURES_MODEL} características, mas a lógica de extração está configurada para 2 (EAR e MAR).")
        print("VOCÊ PRECISA AJUSTAR A FUNÇÃO 'extract_features_from_face_metrics' se seu modelo TCN não foi treinado com EAR e MAR, ou se o número de features for diferente de 2.")
except Exception as e:
    print(f"Erro ao obter input_shape do modelo: {e}")
    print("Não foi possível determinar TIMESTEPS e NUM_FEATURES_MODEL a partir do modelo.")
    exit()

# --- 2. Configuração do MediaPipe Face Mesh ---
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True, # Obtém landmarks mais detalhados para íris, etc.
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# --- 3. Funções Auxiliares para Cálculo de Métricas (EAR e MAR) ---
# Copiadas do seu código LSTM, pois são as features que queremos para o TCN também.

def calculate_distance(point1, point2):
    """Calcula a distância euclidiana entre dois pontos 3D (landmarks)."""
    return math.sqrt((point1.x - point2.x)**2 + (point1.y - point2.y)**2 + (point1.z - point2.z)**2)

def calculate_ear(eye_landmarks, all_landmarks):
    """
    Calcula o Eye Aspect Ratio (EAR) para um olho.
    A fórmula é (dist(P2,P6) + dist(P3,P5)) / (2 * dist(P1,P4)).
    Os índices dos landmarks são baseados na documentação do MediaPipe Face Mesh.
    """
    try:
        # Pontos verticais
        p2 = all_landmarks[eye_landmarks[0]]
        p6 = all_landmarks[eye_landmarks[1]]
        p3 = all_landmarks[eye_landmarks[2]]
        p5 = all_landmarks[eye_landmarks[3]]
        # Pontos horizontais
        p1 = all_landmarks[eye_landmarks[4]]
        p4 = all_landmarks[eye_landmarks[5]]

        vertical_dist1 = calculate_distance(p2, p6)
        vertical_dist2 = calculate_distance(p3, p5)
        horizontal_dist = calculate_distance(p1, p4)

        if horizontal_dist == 0:
            return 0.0
        ear = (vertical_dist1 + vertical_dist2) / (2.0 * horizontal_dist)
        return ear
    except IndexError:
        print("Erro de índice ao acessar landmarks do olho para EAR. Retornando 0.0.")
        return 0.0

def calculate_mar(mouth_landmarks, all_landmarks):
    """
    Calcula o Mouth Aspect Ratio (MAR).
    A fórmula é dist(P_topo, P_base) / dist(P_esquerda, P_direita).
    """
    try:
        p_topo = all_landmarks[mouth_landmarks[0]]
        p_base = all_landmarks[mouth_landmarks[1]]
        p_esquerda = all_landmarks[mouth_landmarks[2]]
        p_direita = all_landmarks[mouth_landmarks[3]]

        vertical_dist = calculate_distance(p_topo, p_base)
        horizontal_dist = calculate_distance(p_esquerda, p_direita)

        if horizontal_dist == 0:
            return 0.0
        mar = vertical_dist / horizontal_dist
        return mar
    except IndexError:
        print("Erro de índice ao acessar landmarks da boca para MAR. Retornando 0.0.")
        return 0.0

# Índices dos landmarks do MediaPipe para olhos e boca
RIGHT_EYE_LANDMARK_INDICES = [160, 144, 158, 153, 33, 133] # P2,P6,P3,P5,P1,P4
LEFT_EYE_LANDMARK_INDICES = [387, 373, 385, 380, 263, 362] # P2,P6,P3,P5,P1,P4
MOUTH_LANDMARK_INDICES = [13, 14, 61, 291] # Topo, Base, Esquerda, Direita (para MAR)


def extract_features_from_face_metrics(face_landmarks_object_list):
    """
    Extrai EAR e MAR a partir dos landmarks faciais.
    Retorna um array NumPy com [ear_medio, mar].
    Esta função espera NUM_FEATURES_MODEL = 2.
    """
    features = np.zeros(NUM_FEATURES_MODEL, dtype=np.float32)

    if face_landmarks_object_list: # Lista de objetos de landmarks faciais detectados
        # Assumindo que estamos processando apenas o primeiro rosto detectado
        all_landmarks = face_landmarks_object_list[0].landmark

        # Calcular EAR para ambos os olhos e tirar a média
        ear_right = calculate_ear(RIGHT_EYE_LANDMARK_INDICES, all_landmarks)
        ear_left = calculate_ear(LEFT_EYE_LANDMARK_INDICES, all_landmarks)
        average_ear = (ear_right + ear_left) / 2.0

        # Calcular MAR
        mar = calculate_mar(MOUTH_LANDMARK_INDICES, all_landmarks)

        if NUM_FEATURES_MODEL == 2:
            features[0] = average_ear
            features[1] = mar
        else:
            print(f"Aviso: 'extract_features_from_face_metrics' gera EAR e MAR, mas 'NUM_FEATURES_MODEL' é {NUM_FEATURES_MODEL}. Isso pode causar incompatibilidade.")
            if NUM_FEATURES_MODEL > 0: features[0] = average_ear
            if NUM_FEATURES_MODEL > 1: features[1] = mar
            # As features restantes seriam zero se NUM_FEATURES_MODEL > 2
            # ou haveria um erro se NUM_FEATURES_MODEL < 2
    return features


# --- 4. Loop Principal da Câmera ---
cap = cv2.VideoCapture(0) # 0 para a câmera padrão
if not cap.isOpened():
    print("Erro: Não foi possível abrir a câmera. Verifique se ela está conectada e disponível.")
    exit()

sequence_buffer = deque(maxlen=TIMESTEPS)
prediction_text = "Aguardando dados..."
prediction_color = (0, 255, 255) # Amarelo

print("\nIniciando captura da câmera. Pressione 'q' para sair.")
print(f"O modelo TCN espera {TIMESTEPS} amostras de tempo (frames) com {NUM_FEATURES_MODEL} características cada.")
if NUM_FEATURES_MODEL == 2:
    print("As características extraídas serão: [EAR médio, MAR].")
else:
    print(f"AVISO: As características extraídas são EAR e MAR, mas o modelo TCN espera {NUM_FEATURES_MODEL}. Verifique a consistência.")


while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("Ignorando frame vazio da câmera.")
        continue

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_rgb.flags.writeable = False # Otimização: tornar a imagem não-escrevível antes do processamento
    results = face_mesh.process(image_rgb)
    image_rgb.flags.writeable = True # Tornar a imagem escrevível novamente para desenhar

    image_output = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR) # Converter de volta para BGR para OpenCV

    current_features = None
    if results.multi_face_landmarks:
        # Desenhar landmarks (opcional)
        for face_landmarks in results.multi_face_landmarks:
            mp_drawing.draw_landmarks(
                image=image_output,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_TESSELATION,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp.solutions.drawing_styles.get_default_face_mesh_tesselation_style())
            mp_drawing.draw_landmarks(
                image=image_output,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_CONTOURS,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp.solutions.drawing_styles.get_default_face_mesh_contours_style())
            
        # Extrair as métricas (EAR, MAR)
        current_features = extract_features_from_face_metrics(results.multi_face_landmarks)
    else:
        # Nenhum rosto detectado, preenche com zeros para manter o tamanho da sequência
        current_features = np.zeros(NUM_FEATURES_MODEL, dtype=np.float32)
        prediction_text = "Nenhum rosto detectado"
        prediction_color = (0,0,255) # Vermelho

    # Adicionar as características atuais ao buffer
    if current_features is not None:
        sequence_buffer.append(current_features)

    # Se o buffer estiver cheio (ou seja, temos TIMESTEPS quadros de dados)
    if len(sequence_buffer) == TIMESTEPS:
        sequence_np = np.array(list(sequence_buffer)) # Shape: (TIMESTEPS, NUM_FEATURES_MODEL)

        # O scaler foi ajustado em dados com shape (num_samples * timesteps, num_features)
        # Para transformar uma única sequência, precisamos achatá-la para (timesteps, num_features)
        sequence_reshaped_for_scaler = sequence_np.reshape(-1, NUM_FEATURES_MODEL)
        
        try:
            if sequence_reshaped_for_scaler.shape[1] != scaler.n_features_in_:
                print(f"Erro crítico de dimensionalidade antes do scaler: Features extraídas ({sequence_reshaped_for_scaler.shape[1]}) != Features esperadas pelo scaler ({scaler.n_features_in_}).")
                print("Verifique NUM_FEATURES_MODEL e a lógica de extract_features_from_face_metrics.")
                prediction_text = "Erro Dim Scaler!"
                prediction_color = (0,0,255) # Vermelho
                sequence_scaled_reshaped = np.zeros_like(sequence_reshaped_for_scaler) # Para evitar crash, mas predição inválida
            else:
                sequence_scaled_reshaped = scaler.transform(sequence_reshaped_for_scaler)
                # Resetar a cor de predição se um erro anterior foi resolvido
                if prediction_text in ["Erro Dim Scaler!", "Erro no Scaler!", "Nenhum rosto detectado"] and current_features.any():
                    prediction_color = (0, 255, 255) # Amarelo para aguardar nova predição
        
        except ValueError as ve:
            print(f"Erro ao aplicar o scaler: {ve}")
            print(f"Scaler esperava {scaler.n_features_in_} features, obteve {sequence_reshaped_for_scaler.shape[1]}.")
            prediction_text = "Erro no Scaler!"
            prediction_color = (0,0,255) # Vermelho
            sequence_scaled_reshaped = np.zeros_like(sequence_reshaped_for_scaler) # Para evitar crash
        
        # Reformatar para (1, TIMESTEPS, NUM_FEATURES_MODEL) para o modelo TCN
        sequence_scaled = sequence_scaled_reshaped.reshape(1, TIMESTEPS, NUM_FEATURES_MODEL)

        # Só fazer a predição se não houve erro grave anterior
        if prediction_text not in ["Erro Dim Scaler!", "Erro no Scaler!", "Nenhum rosto detectado"]:
            prediction_proba = model_cnn_tcn.predict(sequence_scaled, verbose=0)[0] # verbose=0 para não printar progresso
            prediction_class = (prediction_proba > 0.5).astype(int)[0]
            prediction_text = f"Classe: {prediction_class} (Prob: {prediction_proba[0]:.2f})"
            # Verde para classe 1 (Fadigado, por exemplo), Azul claro para 0 (Normal)
            prediction_color = (0, 255, 0) if prediction_class == 1 else (255, 100, 100)
            
    # Mostrar a predição na tela
    cv2.putText(image_output, prediction_text, (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, prediction_color, 2)
    cv2.putText(image_output, f"Buffer: {len(sequence_buffer)}/{TIMESTEPS}", (10, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 0), 1)

    # Mostrar a imagem
    cv2.imshow('Teste Modelo CNN-TCN com Camera', image_output)

    # Pressione 'q' para sair
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

# --- 5. Limpeza ---
cap.release()
cv2.destroyAllWindows()
if 'face_mesh' in locals() and face_mesh is not None:
    face_mesh.close()
print("Teste finalizado.")

2025-05-26 12:56:58.217378: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-26 12:56:58.225372: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748275018.235963   33248 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748275018.238538   33248 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748275018.245357   33248 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

--- Testando o Modelo CNN-TCN com Câmera ---


I0000 00:00:1748275019.707041   33248 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1166 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Ti, pci bus id: 0000:01:00.0, compute capability: 8.9


Modelo CNN-TCN carregado de: processed_data/best_model_cnn_tcn.keras
StandardScaler carregado de: processed_data/scaler_model_cnn_tcn.joblib
Configuração do modelo: TIMESTEPS=90, NUM_FEATURES_MODEL=2

Iniciando captura da câmera. Pressione 'q' para sair.
O modelo TCN espera 90 amostras de tempo (frames) com 2 características cada.
As características extraídas serão: [EAR médio, MAR].


I0000 00:00:1748275020.248924   33248 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1748275020.280716   33452 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 575.51.03), renderer: NVIDIA GeForce RTX 4060 Ti/PCIe/SSE2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1748275020.283053   33435 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1748275020.290310   33433 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1748275020.708543   33433 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
I0000 00:00:1748275026.809869   33395 service.cc:152] XLA service 0x7fbbb8002480 initialized for platform CUDA

Teste finalizado.
